In [4]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [5]:
df = pd.read_excel('./data/redacoes.xlsx')
nota = df['nota1'] + df['nota2'] + df['nota3'] + df['nota4'] + df['nota5']

df['nota'] = nota
tema_e_redacao = "Tema: " + df['tema'] + '\n\n' + "Redacao: " + df['redacoes'] + '\n\n' + "Nota: " + df['nota'].astype(str)
df['tema_e_redacao'] = tema_e_redacao

In [6]:
# Format example content:

all_examples = "\n--------------\n".join(df['tema_e_redacao'].values[:3])

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import List, Dict, Any
from rubrica import *

from dotenv import load_dotenv

load_dotenv()

model_name = "gpt-4o-2024-08-06"
model_name_mini = "gpt-4o-mini"
temperature = 0.3
llm = ChatOpenAI(model_name=model_name_mini, 
                 temperature=temperature)

class GetSchema(BaseModel):
    """Schema das redações"""
    
    tema: str = Field(description="Tema da redação")
    redacao: str = Field(description="Redação")
    nota: int = Field(description="Nota da redação")

def create_model(llm: Any) -> Any:

    system_prompt = """
    Você é um assistente muito prestativo que irá ler várias redações e vai criar novas com base nos exemplos que você viu.
    
    Aqui estão os exemplos:
    
    {all_examples}
    """
    rule_1 = """Você deve criar um tema, criar a redação e dar uma nota para ela. A rubrica para dar as notas é a seguinte:
    {rubrica}
    """

    prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt), 
                ("system", rule_1),
                # ("human", "Aqui está o tema gerado + redação gerada + nota: \n\n"),
            ]
    )

    llm_with_tools = llm.bind_tools([GetSchema]) #, strict=True)
    chain = prompt | llm_with_tools

    return chain

In [8]:
chain = create_model(llm)

In [9]:
notas_possiveis = list(range(650, 1050, 50))

notas_exemplos = [np.random.choice(notas_possiveis) for _ in range(1000)]

In [15]:
results = []
for n in tqdm(notas_exemplos[:200]):
    try:
        response = chain.invoke({"all_examples": all_examples, 
                                "rubrica": f"Use o bom senso para dar notas de 0 a 1000. A nota deve ser um número inteiro. OBS.: Faça uma redação que dê uma nota: {n}. OBS2.: Faça um parágrafo só."})
        result = response.tool_calls[0]['args']
        results.append(result)
    except Exception as e:
        print(e)
        continue

  2%|▏         | 3/200 [00:16<17:00,  5.18s/it]

list index out of range


  2%|▎         | 5/200 [00:26<16:29,  5.07s/it]

list index out of range


  3%|▎         | 6/200 [00:30<15:13,  4.71s/it]

...

 96%|█████████▋| 193/200 [18:24<00:31,  4.47s/it]

list index out of range


 98%|█████████▊| 196/200 [18:38<00:17,  4.34s/it]

list index out of range


100%|██████████| 200/200 [18:52<00:00,  5.66s/it]


In [17]:
len(results)

115

In [18]:
results_content = [f"Tema: {r['tema']}\n\nRedação: {r['redacao']}\n\nNota: {r['nota']}" for r in results]

In [23]:
dict_result = {'tema': [r['tema'] for r in results], 
               'redacao': [r['redacao'] for r in results], 
               'nota': [r['nota'] for r in results]}
df_sintetico = pd.DataFrame(dict_result)

In [24]:
df_sintetico.to_csv('./data/redacoes_sinteticas.csv', index=False)

In [25]:
df_sintetico

,tema,redacao,nota
0,A importância da educação financeira nas escol...,A educação financeira é uma ferramenta essenci...,650
1,A importância da educação financeira nas escol...,Em um mundo cada vez mais globalizado e dinâmi...,1000
2,Impacto das redes sociais na saúde mental dos ...,"Na contemporaneidade, as redes sociais emergem...",1000
3,Impacto da tecnologia na educação: desafios e ...,A tecnologia tem se tornado uma aliada indispe...,850
4,A importância da educação financeira nas escol...,A educação financeira é uma ferramenta essenci...,1000
...,...,...,...
110,A importância da educação financeira nas escol...,A educação financeira é uma ferramenta essenci...,800
111,Impacto das redes sociais na saúde mental dos ...,"Em um mundo cada vez mais conectado, as redes ...",900
112,Impacto das redes sociais na saúde mental dos ...,"No contexto contemporâneo, as redes sociais em...",750
113,A importância da educação financeira nas escol...,A educação financeira é uma ferramenta essenci...,800


In [29]:
df_sintetico['tema_e_redacao'] = "Tema: " + df_sintetico['tema'] + '\n\n' + "Redação: " + df_sintetico['redacao']
df_sintetico[['tema_e_redacao', 'nota']]

,tema_e_redacao,nota
0,Tema: A importância da educação financeira nas...,650
1,Tema: A importância da educação financeira nas...,1000
2,Tema: Impacto das redes sociais na saúde menta...,1000
3,Tema: Impacto da tecnologia na educação: desaf...,850
4,Tema: A importância da educação financeira nas...,1000
...,...,...
110,Tema: A importância da educação financeira nas...,800
111,Tema: Impacto das redes sociais na saúde menta...,900
112,Tema: Impacto das redes sociais na saúde menta...,750
113,Tema: A importância da educação financeira nas...,800


In [32]:
df_real = pd.read_excel('./data/redacoes.xlsx')
df_real['tema_e_redacao'] = "Tema: " + df_real['tema'] + '\n\n' + "Redacao: " + df_real['redacoes'] 
df_real['nota'] = df_real['nota1'] + df_real['nota2'] + df_real['nota3'] + df_real['nota4'] + df_real['nota5']
df_real[['tema_e_redacao', 'nota']]

,tema_e_redacao,nota
0,"Tema: ""Invisibilidade e registro civil: garant...",920
1,"Tema: ""Democratização do acesso ao cinema no B...",980
2,"Tema: ""Desafios para o enfrentamento da invisi...",900
3,"Tema: ""Caminhos para combater a intolerância r...",680
4,"Tema: ""Invisibilidade e registro civil: garant...",960
5,Tema: “O estigma associado às doenças mentais ...,700
6,"Tema: ""Desafios para a valorização de comunida...",920
7,Tema: “O estigma associado às doenças mentais ...,900
8,"Tema: ""Desafios para a valorização de comunida...",840
9,Tema: “O estigma associado às doenças mentais ...,340


In [33]:
df_to_train = pd.concat([df_real[['tema_e_redacao', 'nota']], df_sintetico[['tema_e_redacao', 'nota']]], axis=0)
df_to_train

,tema_e_redacao,nota
0,"Tema: ""Invisibilidade e registro civil: garant...",920
1,"Tema: ""Democratização do acesso ao cinema no B...",980
2,"Tema: ""Desafios para o enfrentamento da invisi...",900
3,"Tema: ""Caminhos para combater a intolerância r...",680
4,"Tema: ""Invisibilidade e registro civil: garant...",960
...,...,...
110,Tema: A importância da educação financeira nas...,800
111,Tema: Impacto das redes sociais na saúde menta...,900
112,Tema: Impacto das redes sociais na saúde menta...,750
113,Tema: A importância da educação financeira nas...,800


In [34]:
df_to_train.to_csv('./data/redacoes_to_train.csv')